In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
from pathlib import Path

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier

from collections import defaultdict

In [ ]:
df = pd.read_csv("analysis/not_uploaded/IA/AVA/.scratch.ckpts.IA.pexels.scores-one.change_regress.epoch-9.pth.txt")
df["img"] = df["img"].apply(lambda row: int(Path(row).stem))
df["score"] = df["score"].apply(lambda row: eval(row)[0])

with open("analysis/not_uploaded/IA/AVA/style_image_lists/styles.txt") as f:
    styles_dict = [(int(line.split(" ")[0]), line.split(" ")[1].strip()) for line in f.readlines()]
    styles_dict = dict(styles_dict)

with open("analysis/not_uploaded/IA/AVA/style_image_lists/train.jpgl") as f:
    train_files = [int(line) for line in f.readlines()]

with open("analysis/not_uploaded/IA/AVA/style_image_lists/train.lab") as f:
    train_label = [int(line) for line in f.readlines()]

train_files = defaultdict(lambda: -1, zip(train_files, train_label))

with open("analysis/not_uploaded/IA/AVA/style_image_lists/test.jpgl") as f:
    test_files = [int(line) for line in f.readlines()]

with open("analysis/not_uploaded/IA/AVA/style_image_lists/test.multilab") as f:
    test_label = [line.strip().split(" ") for line in f.readlines()]
    test_label = [[int(lab) for lab in line] for line in test_label]

test_files = defaultdict(lambda: [], zip(test_files, test_label))

df = df[df["img"].isin(train_files) | df["img"].isin(test_files)]
df["mode"] = df["img"].apply(lambda row: "train" if row in train_files else "test")

df["train_label"] = df["img"].apply(lambda row: train_files[row])
df["test_label"] = df["img"].apply(lambda row: test_files[row])

df

In [ ]:
features = [c for c in df.columns if c not in ["index", "img"]]
reg_df = df.dropna()
train = reg_df[reg_df["mode"]=="train"].reset_index()[features]
test = reg_df[reg_df["mode"]=="test"].reset_index()[features]
features

In [ ]:
reg = OneVsRestClassifier(svm.LinearSVC(class_weight='balanced'), n_jobs=-1).fit(train.drop(columns=["mode", "train_label", "test_label"]), train["train_label"])
reg.score(train.drop(columns=["mode", "train_label", "test_label"]), train["train_label"])

In [ ]:
preds = []
for est in reg.estimators_:
    preds.append(est.predict(test.drop(columns=["mode", "train_label", "test_label", "pred", "y_pred"])))

In [ ]:
y_pred = [[] for _ in range(len(preds[0]))]
for est in preds:
    for i, row in enumerate(est):
        y_pred[i].append(row)
test["y_pred"] = y_pred

In [ ]:
test["pred"] = reg.predict(test.drop(columns=["mode", "train_label", "test_label"]))

In [ ]:
test[["test_label", "y_pred"]]

In [ ]:
test["found"] = test.apply(lambda row: row.test_label[row.pred-1], axis=1)

In [ ]:
sum(list(test["found"]))/len(list(test["found"]))

In [ ]:
test["pred"] = clf.predict(test.drop(columns=["mode", "train_label", "test_label", "pred", "found"]))

In [ ]:
probs = clf.predict_proba(test.drop(columns=["mode", "train_label", "test_label", "pred", "found"]))

In [ ]:
probs2 = [[1 if score > 1/14 else 0 for score in i] for i in probs]

In [ ]:
test[["test_label", "pred"]]

In [ ]:
precs = []
for i in range(14):
    precs.append(sklearn.metrics.precision_score(test["test_label"].apply(lambda row: row[i]), test["y_pred"].apply(lambda row: row[i])))
    print(precs[-1])
print("avg",sum(precs) / len(precs))

In [ ]:
test["found"] = test.apply(lambda row: row.test_label[row.pred-1], axis=1)

In [ ]:
sum(list(test["found"]))/len(list(test["found"]))